# from情報を登録するためのデータテーブル作成

In [2]:
import pandas as pd
import numpy as np

# 不要な警告文を非表示にする
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
# 今日の日付を取得
today = datetime.today().date().strftime('%Y%m%d')

In [3]:
# チームごとの勝敗履歴が登録されている(form：右にあるデータが最新)
df_team_status = pd.read_csv('statistics/team/team_statistics_2023.csv')
# playedが最大かつteam.idが一意になるように変更
df_team_status = df_team_status.loc[df_team_status.groupby('team.id')['played'].idxmax()].\
                                    rename(columns={'team.id':'team_id'}).\
                                    reset_index(drop=True)
print(df_team_status.shape)
df_team_status.head(3)

(18, 127)


Unnamed: 0                              form  league.id league.name  \
0           0  DLDWDLWLDWLWWWLWDLDLLLDLDWLDLLWD         98   J1 League   
1           3  DDLLLLWDLDWDLDLLLDLDLWDDDWWLWLDD         98   J1 League   
2           8  DLDWWWWWLWWLLWLWLLLDDLWWDWWLWDWD         98   J1 League   

  league.country                                        league.logo  \
0          Japan  https://media-4.api-sports.io/football/leagues...   
1          Japan  https://media-4.api-sports.io/football/leagues...   
2          Japan  https://media-4.api-sports.io/football/leagues...   

                                  league.flag  league.season  team_id  \
0  https://media-4.api-sports.io/flags/jp.svg           2023      279   
1  https://media-4.api-sports.io/flags/jp.svg           2023      281   
2  https://media-4.api-sports.io/flags/jp.svg           2023      282   

             team.name  ... cards.red.76-90.total  cards.red.76-90.percentage  \
0    Consadole Sapporo  ...                   NaN                         NaN   
1       Kashiwa Reysol  ...                   2.0                      66.67%   
2  Sanfrecce Hiroshima  ...                   NaN                         NaN   

   cards.red.91-105.total  cards.red.91-105.percentage  \
0                     NaN                          NaN   
1                     NaN                          NaN   
2                     NaN                          NaN   

   cards.red.106-120.total  cards.red.106-120.percentage  cards.red..total  \
0                      NaN                           NaN               NaN   
1                      NaN                           NaN               NaN   
2                      NaN                           NaN               NaN   

   cards.red..percentage  formation  played  
0                    NaN    3-4-2-1      27  
1                    NaN      4-4-2      19  
2                    NaN    3-4-2-1      28  

[3 rows x 127 columns]

In [4]:
# 各試合の情報がチームごと,日付ごと、その他諸々ごとに登録されている。チームごと、日付事にソートする
df_fixtures = pd.read_csv('fixtures/fixtures_2023.csv')
# 日付データの型変換
# dtを加えることで列全体に対して任意の文字列形式変換が可能になる
df_fixtures['fixture.date'] = pd.to_datetime(df_fixtures['fixture.date']).dt.strftime('%Y%m%d')
#チームごと、日付事にソートする
#後々、df_fixturesとteam.idをキーにして結合、カラム名も変更する
df_fixtures = \
    df_fixtures[['teams.home.id','teams.away.id','fixture.id','fixture.date']].\
    rename(columns={'teams.home.id':'home_id','teams.away.id':'away_id'}).\
    sort_values(['home_id','away_id','fixture.date']).reset_index(drop=True)
print(df_fixtures.shape)
#j1リーグの全試合日程と、対戦の組み合わせがデータフレームに格納される
df_fixtures.head(3)

(306, 4)


home_id  away_id  fixture.id fixture.date
0      279      281      994863     20230930
1      279      282      994890     20231111
2      279      284      994845     20230916

In [5]:
# fixture.idが一意であるか確認する
print(len(df_fixtures['fixture.id']))
print(len(df_fixtures['fixture.id'].unique()))

306
306


### todayを検証したいリーグ戦の前日にすると検証したい日程までの戦績をとれるようになる

In [6]:
# # 試合開催日と今の日付を比較して、今の日付のほうが大きければFalse、そうでなければTrueを入れる
df_fixtures["game_flg"] = df_fixtures['fixture.date'] < today
df_fixtures.head(3)

home_id  away_id  fixture.id fixture.date  game_flg
0      279      281      994863     20230930      True
1      279      282      994890     20231111      True
2      279      284      994845     20230916      True

### 以下練習用（１チームあたりで取得できるかテスト）

In [7]:
# # 1チーム分（teamid=279）の試合条件を取得（ホーム戦時の戦績＆アウェイ戦時の戦績、対戦チームの戦績）
# df_test_279 = df_fixtures[(df_fixtures['home_id'] == 279) | (df_fixtures['away_id'] == 279)]

# print(len(df_test_279))
# df_test_279.head(3)

In [8]:
# #現状試合結果がある部分までのデータを取得する。
# df_test_279_end = df_test_279[df_test_279['game_flg'] == True]
# #確認用：現時点で開催済みの試合数
# print(len(df_test_279_end))
# # 試合日程時点での戦績のカラムを作成する。
# df_test_279_end['form_ago_279'] = 0
# # fixture.dateの順に並び変える。indexを初期化しないと、formから適切なindexが指定できない
# df_test_279_end = df_test_279_end.sort_values('fixture.date').reset_index(drop=True)
# #　以下で、id = 279の今日までの試合日程のデータフレームを確認する
# df_test_279_end.head(3)

In [9]:
# # 抽出したデータフレームの長さが
# # 時系列でテーブルが並べられている。
# # form列から古い順に(左から取得)、スライスを用いて、一試合前までの戦績を抽出する。
# for i in range(len(df_test_279_end)):
#     if i == 0:
#         df_test_279_end.loc[i,'form_ago_279'] = '0'
#     else :
#         if df_test_279_end['home_id'][i] == 279:
#             df_test_279_end.loc[i,'form_ago_279'] =  df_team_status['form'][0][:i]
#         else :
#             df_test_279_end.loc[i,'form_ago_279'] =  df_team_status['form'][0][:i]
# df_test_279_end.head(5)

できたのでテスト終了
以下、すべてのデータに対して適合するかテスト

---

## チームごとにデータフレームを作成する

In [10]:
# home_id_listの要素。要素値、対戦チームと試合日程・試合開催フラグを格納したデータフレーム、今日までの戦績を持ったデータフレームを登録する
def process_team_data(ele, 
                      team_id, 
                      df_fixtures, 
                      df_team_status):
    
    # team_idとして渡されたチームのインデックスをdf_fixturesから取得（ホーム戦時の戦績＆アウェイ戦時の戦績、対戦チームの戦績）
    df_team = df_fixtures[(df_fixtures['home_id'] == team_id) | (df_fixtures['away_id'] == team_id)]

    # # 試合日程時点での戦績のカラムを作成する。
    # fixture.dateの順に並び変える。indexを初期化しないと、indexが指定できない
    # 試合の順番で並び替えることで、df_team_statusのformカラムから、
    #１試合目~~対象のインデックス番号までの長さでformを抽出できる。
    df_team = df_team.sort_values('fixture.date').reset_index(drop=True)

    # form列から古い順に(左から取得)、スライスを用いて、一試合前までの戦績を抽出する。
    # 1試合目にはformが存在しないため、1から開始した
    for i in range(1,len(df_team)):
        # L7で渡されたチームがホーム戦であれば、ホームのフォームにそのチームの戦績を登録する
        if team_id == df_team.loc[i,'home_id']:
            df_team.loc[i, 'form_home'] = df_team_status['form'][ele][:i]
        # その逆
        else :
            df_team.loc[i, 'form_away'] = df_team_status['form'][ele][:i]

    return df_team

In [11]:
# リーグ戦に所属するteamIDを取得する
home_id_list = df_fixtures['home_id'].unique()

# 関数からデータフレームを受け取る。データフレームの名前は動的に変更できない。
# 結果を辞書に格納することで、補完する。（要素名なら変更できるので）
results = {}
# ホームチーム、アウェイチームの試合開催日時点のフォームを登録するためのカラムの生成
# df_fixtures[['form_home','form_away']] = '0'

# チーム数の数だけ関数を呼び出す
for element, team_id in enumerate(home_id_list):
    # チーム名を要素名にして返却されたデータフレームを受けとる
    results[team_id] = process_team_data(element, team_id, df_fixtures, df_team_status)

# キーに結合されたデータフレームをすべて縦に結合する
df_form = pd.concat(results.values(), ignore_index=True)
df_form

home_id  away_id  fixture.id fixture.date  game_flg  \
0        282      279      982850     20230218      True   
1        279      289      994622     20230225      True   
2        311      279      994632     20230304      True   
3        279      296      994641     20230312      True   
4        293      279      994651     20230318      True   
..       ...      ...         ...          ...       ...   
607      294      316      994873     20231020      True   
608      316      296      994889     20231028      True   
609      293      316      994897     20231111      True   
610      287      316      994899     20231125     False   
611      316      282      994915     20231203     False   

                            form_home                         form_away  
0                                 NaN                               NaN  
1                                   D                               NaN  
2                                 NaN                                DL  
3                                 DLD                               NaN  
4                                 NaN                              DLDW  
..                                ...                               ...  
607                               NaN     LWWDWDWLDLWLDDLLLLWWWWWLLWWWD  
608    LWWDWDWLDLWLDDLLLLWWWWWLLWWWDL                               NaN  
609                               NaN   LWWDWDWLDLWLDDLLLLWWWWWLLWWWDLL  
610                               NaN  LWWDWDWLDLWLDDLLLLWWWWWLLWWWDLLW  
611  LWWDWDWLDLWLDDLLLLWWWWWLLWWWDLLW                               NaN  

[612 rows x 7 columns]

In [12]:
# 重複するhome_idとaway_idの組み合わせを持つ行を見つける
# duplicated : 指定したカラムの重複するインデックスを取得できる.keepをFalseで指定することで、重複する行をすべて取得する
duplicates = df_form[df_form.duplicated(subset=['home_id', 'away_id'], keep=False)].sort_values('fixture.id')
# # NaNでない値でNaNを補完する.
## ffill() : ひとつ後のインデックスの欠損値を現在の値で補完する
## bfill():ひとつ前のインデックスの欠損値を補完する
duplicates_combined = duplicates.groupby(['home_id', 'away_id']).apply(lambda x: x.ffill().bfill()).drop_duplicates()
# 結果を確認する
duplicates_combined.sort_values('fixture.id').reset_index(drop=True).to_csv('fixture_form.csv', index=False)